In [ ]:
#%%jupyter_template init
from jupyter_template import magic
magic.init(lambda _=globals: _())

# Enrichr Figure 

This template creates a publishable figure summarizing enrichment analysis results from Enrichr. 

The resulting figure will contain the top 5 enriched terms and their corresponding p-values for each library chosen. Red bars correspond to terms with significant p-values (<0.05). An asterisk (*) next to a p-value indicates the term also has a significant adjusted p-value (<0.05).

In [ ]:
import pandas as pd 
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator
import datetime
from datetime import date

In [ ]:
%%jupyter_template hide_code

{% do SectionField(name='section1', title = 'Publish Your Enrichr Results', subtitle = 'Create a publishable figure summarizing your Enrichr analysis results. The resulting figure will contain the top 5 enriched terms and their corresponding p-values for each library chosen. Red bars correspond to terms with significant p-values (<0.05). An asterisk (*) next to a p-value indicates the term also has a significant adjusted p-value (<0.05).', img = 'enrichr-icon.png')%}


### Input options

In [ ]:
%%jupyter_template code_eval

gene_list_filename = "{{ FileField(name='gene_list_filename', label='Gene List File', default='', description='Upload your gene list as a text file. One gene per row ONLY.',section = 'section1').value }}"

enrichr_libraries = {{ MultiChoiceField(name='enrichr_libraries', description='Select the Enrichr libraries you would like in your figure.', label='Enrichr Libraries', default=[], section = 'section1',choices=[
    'Genes_Associated_with_NIH_Grants',
    'Cancer_Cell_Line_Encyclopedia',
    'Achilles_fitness_decrease',
    'Achilles_fitness_increase',
    'Aging_Perturbations_from_GEO_down',
    'Aging_Perturbations_from_GEO_up',
    'Allen_Brain_Atlas_down',
    'Allen_Brain_Atlas_up',
    'ARCHS4_Cell-lines',
    'ARCHS4_IDG_Coexp',
    'ARCHS4_Kinases_Coexp',
    'ARCHS4_TFs_Coexp',
    'ARCHS4_Tissues',
    'BioCarta_2013',
    'BioCarta_2015',
    'BioCarta_2016',
    'BioPlanet_2019',
    'BioPlex_2017',
    'CCLE_Proteomics_2020',
    'ChEA_2013',
    'ChEA_2015',
    'ChEA_2016',
    'Chromosome_Location',
    'Chromosome_Location_hg19',
    'ClinVar_2019',
    'CORUM',
    'Data_Acquisition_Method_Most_Popular_Genes',
    'dbGaP',
    'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019',
    'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019',
    'Disease_Perturbations_from_GEO_down',
    'Disease_Perturbations_from_GEO_up',
    'Disease_Signatures_from_GEO_down_2014',
    'Disease_Signatures_from_GEO_up_2014',
    'DisGeNET',
    'Drug_Perturbations_from_GEO_2014',
    'Drug_Perturbations_from_GEO_down',
    'Drug_Perturbations_from_GEO_up',
    'DrugMatrix',
    'DSigDB',
    'Elsevier_Pathway_Collection',
    'ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X',
    'ENCODE_Histone_Modifications_2013',
    'ENCODE_Histone_Modifications_2015',
    'ENCODE_TF_ChIP-seq_2014',
    'ENCODE_TF_ChIP-seq_2015',
    'Enrichr_Libraries_Most_Popular_Genes',
    'Enrichr_Submissions_TF-Gene_Coocurrence',
    'Epigenomics_Roadmap_HM_ChIP-seq',
    'ESCAPE',
    'Gene_Perturbations_from_GEO_down',
    'Gene_Perturbations_from_GEO_up',
    'GeneSigDB',
    'Genome_Browser_PWMs',
    'GO_Biological_Process_2013',
    'GO_Biological_Process_2015',
    'GO_Biological_Process_2017',
    'GO_Biological_Process_2017b',
    'GO_Biological_Process_2018',
    'GO_Cellular_Component_2013',
    'GO_Cellular_Component_2015',
    'GO_Cellular_Component_2017',
    'GO_Cellular_Component_2017b',
    'GO_Cellular_Component_2018',
    'GO_Molecular_Function_2013',
    'GO_Molecular_Function_2015',
    'GO_Molecular_Function_2017',
    'GO_Molecular_Function_2017b',
    'GO_Molecular_Function_2018',
    'GTEx_Tissue_Sample_Gene_Expression_Profiles_down',
    'GTEx_Tissue_Sample_Gene_Expression_Profiles_up',
    'GWAS_Catalog_2019',
    'HMDB_Metabolites',
    'HMS_LINCS_KinomeScan',
    'HomoloGene',
    'Human_Gene_Atlas',
    'Human_Phenotype_Ontology',
    'HumanCyc_2015',
    'HumanCyc_2016',
    'huMAP',
    'InterPro_Domains_2019',
    'Jensen_COMPARTMENTS',
    'Jensen_DISEASES',
    'Jensen_TISSUES',
    'KEA_2013',
    'KEA_2015',
    'KEGG_2013',
    'KEGG_2015',
    'KEGG_2016',
    'KEGG_2019_Human',
    'KEGG_2019_Mouse',
    'Kinase_Perturbations_from_GEO_down',
    'Kinase_Perturbations_from_GEO_up',
    'L1000_Kinase_and_GPCR_Perturbations_down',
    'L1000_Kinase_and_GPCR_Perturbations_up',
    'Ligand_Perturbations_from_GEO_down',
    'Ligand_Perturbations_from_GEO_up',
    'LINCS_L1000_Chem_Pert_down',
    'LINCS_L1000_Chem_Pert_up',
    'LINCS_L1000_Ligand_Perturbations_down',
    'LINCS_L1000_Ligand_Perturbations_up',
    'lncHUB_lncRNA_Co-Expression',
    'MCF7_Perturbations_from_GEO_down',
    'MCF7_Perturbations_from_GEO_up',
    'MGI_Mammalian_Phenotype_2013',
    'MGI_Mammalian_Phenotype_2017',
    'MGI_Mammalian_Phenotype_Level_3',
    'MGI_Mammalian_Phenotype_Level_4',
    'MGI_Mammalian_Phenotype_Level_4_2019',
    'Microbe_Perturbations_from_GEO_down',
    'Microbe_Perturbations_from_GEO_up',
    'miRTarBase_2017',
    'Mouse_Gene_Atlas',
    'MSigDB_Computational',
    'MSigDB_Oncogenic_Signatures',
    'NCI-60_Cancer_Cell_Lines',
    'NCI-Nature_2015',
    'NCI-Nature_2016',
    'NIH_Funded_PIs_2017_AutoRIF_ARCHS4_Predictions',
    'NIH_Funded_PIs_2017_GeneRIF_ARCHS4_Predictions',
    'NIH_Funded_PIs_2017_Human_AutoRIF',
    'NIH_Funded_PIs_2017_Human_GeneRIF',
    'NURSA_Human_Endogenous_Complexome',
    'Old_CMAP_down',
    'Old_CMAP_up',
    'OMIM_Disease',
    'OMIM_Expanded',
    'Panther_2015',
    'Panther_2016',
    'Pfam_Domains_2019',
    'Pfam_InterPro_Domains',
    'PheWeb_2019',
    'Phosphatase_Substrates_from_DEPOD',
    'PPI_Hub_Proteins',
    'ProteomicsDB_2020',
    'Rare_Diseases_AutoRIF_ARCHS4_Predictions',
    'Rare_Diseases_AutoRIF_Gene_Lists',
    'Rare_Diseases_GeneRIF_ARCHS4_Predictions',
    'Rare_Diseases_GeneRIF_Gene_Lists',
    'Reactome_2013',
    'Reactome_2015',
    'Reactome_2016',
    'RNA-Seq_Disease_Gene_and_Drug_Signatures_from_GEO',
    'SILAC_Phosphoproteomics',
    'SubCell_BarCode',
    'SysMyo_Muscle_Gene_Sets',
    'TargetScan_microRNA',
    'TargetScan_microRNA_2017',
    'TF-LOF_Expression_from_GEO',
    'TF_Perturbations_Followed_by_Expression',
    'Tissue_Protein_Expression_from_Human_Proteome_Map',
    'Tissue_Protein_Expression_from_ProteomicsDB',
    'Transcription_Factor_PPIs',
    'TRANSFAC_and_JASPAR_PWMs',
    'TRRUST_Transcription_Factors_2019',
    'UK_Biobank_GWAS_v1',
    'Virus-Host_PPI_P-HIPSTer_2020',
    'Virus_Perturbations_from_GEO_down',
    'Virus_Perturbations_from_GEO_up',
    'VirusMINT',
    'WikiPathways_2013',
    'WikiPathways_2015',
    'WikiPathways_2016',
    'WikiPathways_2019_Human',
    'WikiPathways_2019_Mouse']).value}}


figure_file_format = "{{ ChoiceField(name='figure_file_format', label='File Format', choices = ['PNG', 'SVG', 'PDF'], default=['PNG'], description='Choose the format to save your figure.', section = 'section1').value }}"

output_file_name = "{{ StringField(name='output_file_name', label='File Name (Optional)', default='', description='Enter a name/description to save your figure.', section = 'section1').safe_value }}"

annot_dict_filename = "{{ FileField(name='annot_dict_filename', label='Term Conversions (Optional)', default=' ', description='Enrichment terms may be too long for the figure and may need to be manually changed. You may upload a TSV file with two columns containing original terms (left column) and your corresponding preferred shortened terms (right column).', section = 'section1').value }}"


 

### Import gene list

In [ ]:
# Import gene list file 
open_gene_list_file = open(gene_list_filename,'r')
lines = open_gene_list_file.readlines()
genes = [x.strip() for x in lines]
open_gene_list_file.close()

### Import term conversion file (Optional)

In [ ]:
# Conversion dictionary for long terms
annot_dict = {}

# Import term conversion file if one is specified 
if annot_dict_filename != '':
    open_conversion_file = open(annot_dict_filename,'r')
    lines = open_conversion_file.readlines()
    for line in lines:
        line = line.strip()
        line = line.split('\t')
        annot_dict[line[0]]=line[1]
    open_conversion_file.close()

### Get Enrichr Results

In [ ]:
# Function to get Enrichr Results 
# Takes a gene list and Enrichr libraries as input 
def Enrichr_API(enrichr_gene_list, all_libraries):


    all_terms = []
    all_pvalues =[] 
    all_adjusted_pvalues = []


    for library_name in all_libraries : 
        ENRICHR_URL = 'http://amp.pharm.mssm.edu/Enrichr/addList'
        genes_str = '\n'.join(enrichr_gene_list)
        description = 'Example gene list'
        payload = {
            'list': (None, genes_str),
            'description': (None, description)
        }

        response = requests.post(ENRICHR_URL, files=payload)
        if not response.ok:
            raise Exception('Error analyzing gene list')

        data = json.loads(response.text)

        ENRICHR_URL = 'http://amp.pharm.mssm.edu/Enrichr/enrich'
        query_string = '?userListId=%s&backgroundType=%s'
        user_list_id = data['userListId']
        gene_set_library = library_name
        response = requests.get(
            ENRICHR_URL + query_string % (user_list_id, gene_set_library)
         )
        if not response.ok:
            raise Exception('Error fetching enrichment results')

        data = json.loads(response.text)

        results_df  = pd.DataFrame(data[library_name][0:5])
        all_terms.append(list(results_df[1]))
        all_pvalues.append(list(results_df[2]))
        all_adjusted_pvalues.append(list(results_df[6]))
    
    return([all_terms,all_pvalues,all_adjusted_pvalues])



 

In [ ]:
# get enrichr results
results = Enrichr_API(genes,enrichr_libraries)

### Plot Enrichr Results 

In [ ]:
# Function plots results 
# Takes all terms, all p-values, all adjusted p-values, plot title, Enrichr libraries , and specified figure format
def enrichr_figure(all_terms,all_pvalues, all_adjusted_pvalues, plot_name, all_libraries, fig_format): 
    
    # rows and columns depend on number of Enrichr libraries submitted 
    rows = []
    cols = []
    
    # If only 1 Enrichr library selected, make simple plot 
    if len(all_libraries)==1:
        #fig,axes = plt.subplots(1, 1,figsize=[8.5,6])
        plt.figure(figsize=(12,6))
        rows = [0]
        cols = [0]
        i = 0 
        bar_colors = ['tomato' if (x < 0.05) else 'lightgrey' for x in all_pvalues[i]]
        fig = sns.barplot(x=np.log10(all_pvalues[i])*-1, y=all_terms[i], palette=bar_colors)
        fig.axes.get_yaxis().set_visible(False)
        fig.set_title(all_libraries[i].replace('_',' '),fontsize=26)
        fig.set_xlabel('-Log10(p-value)',fontsize=25)
        fig.xaxis.set_major_locator(MaxNLocator(integer=True))
        fig.tick_params(axis='x', which='major', labelsize=20)
        if max(np.log10(all_pvalues[i])*-1)<1:
            fig.xaxis.set_ticks(np.arange(0, max(np.log10(all_pvalues[i])*-1), 0.1))
        for ii,annot in enumerate(all_terms[i]):
            if annot in annot_dict.keys():
                annot = annot_dict[annot]
            if all_adjusted_pvalues[i][ii] < 0.05:
                annot = '  *'.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))]) 
            else:
                annot = '  '.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))])

            title_start= max(fig.axes.get_xlim())/200
            fig.text(title_start,ii,annot,ha='left',wrap = True, fontsize = 26)
            fig.patch.set_edgecolor('black')  
            fig.patch.set_linewidth('2')
        
    
    # If there are an even number of Enrichr libraries below 6
    # Plots 1x2 or 2x2
    else:
        if len(all_libraries) % 2 == 0 and len(all_libraries) < 5:
                for i in range(0,int(len(all_libraries)/2)):    
                    rows = rows + [i]*2
                    cols = list(range(0,2))*int(len(all_libraries)/2)    
                fig, axes = plt.subplots(len(np.unique(rows)), len(np.unique(cols)),figsize=[7,int(2* len(np.unique(rows)))]) 
    
        
        # All other # of libraries 6 and above will have 3 columns and a flexible number of rows to accomodate all plots
        else:
            for i in range(0,int(np.ceil(len(all_libraries)/3))):
                rows = rows + [i]*3
                cols = list(range(0,3))*int(np.ceil(len(all_libraries)/3))
            fig, axes = plt.subplots(len(np.unique(rows)), len(np.unique(cols)),figsize=[8,int(2* len(np.unique(rows)))])
           
        # If final figure only has one row...
        if len(np.unique(rows))==1:
            for i,library_name in enumerate(all_libraries):
                bar_colors = ['tomato' if (x < 0.05) else 'lightgrey' for x in all_pvalues[i]]
                sns.barplot(x=np.log10(all_pvalues[i])*-1, y=all_terms[i],ax=axes[i], palette=bar_colors)
                axes[i].axes.get_yaxis().set_visible(False)
                axes[i].set_title(library_name.replace('_',' '),fontsize=36)
                axes[i].set_xlabel('-Log10(p-value)',fontsize=35)
                axes[i].xaxis.set_major_locator(MaxNLocator(integer=True))
                axes[i].tick_params(axis='x', which='major', labelsize=30)
                if max(np.log10(all_pvalues[i])*-1)<1:
                    axes[i].xaxis.set_ticks(np.arange(0, max(np.log10(all_pvalues[i])*-1), 0.1))
                for ii,annot in enumerate(all_terms[i]):
                    if annot in annot_dict.keys():
                        annot = annot_dict[annot]
                    if all_adjusted_pvalues[i][ii] < 0.05:
                        annot = '  *'.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))]) 
                    else:
                        annot = '  '.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))])

                    title_start= max(axes[i].axes.get_xlim())/200
                    axes[i].text(title_start,ii,annot,ha='left',wrap = True, fontsize = 36)
                    axes[i].patch.set_edgecolor('black')  
                    axes[i].patch.set_linewidth('2')

            plt.subplots_adjust(top=4.5, right = 4.7,wspace = 0.03,hspace = 0.2)


        # If the final figure has more than one row...
        else:


            for i,library_name in enumerate(all_libraries):
                bar_colors = ['tomato' if (x < 0.05) else 'lightgrey' for x in all_pvalues[i]]
                sns.barplot(x=np.log10(all_pvalues[i])*-1, y=all_terms[i],ax=axes[rows[i],cols[i]], palette=bar_colors)
                axes[rows[i],cols[i]].axes.get_yaxis().set_visible(False)
                axes[rows[i],cols[i]].set_title(library_name.replace('_',' '),fontsize=36)
                axes[rows[i],cols[i]].set_xlabel('-Log10(p-value)',fontsize=35)
                axes[rows[i],cols[i]].xaxis.set_major_locator(MaxNLocator(integer=True))
                axes[rows[i],cols[i]].tick_params(axis='x', which='major', labelsize=30)
                if max(np.log10(all_pvalues[i])*-1)<1:
                    axes[rows[i],cols[i]].xaxis.set_ticks(np.arange(0, max(np.log10(all_pvalues[i])*-1), 0.1))
                for ii,annot in enumerate(all_terms[i]):
                    if annot in annot_dict.keys():
                        annot = annot_dict[annot]
                    if all_adjusted_pvalues[i][ii] < 0.05:
                        annot = '  *'.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))]) 
                    else:
                        annot = '  '.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))])

                    title_start= max(axes[rows[i],cols[i]].axes.get_xlim())/200
                    axes[rows[i],cols[i]].text(title_start,ii,annot,ha='left',wrap = True, fontsize = 36)
                    axes[rows[i],cols[i]].patch.set_edgecolor('black')  
                    axes[rows[i],cols[i]].patch.set_linewidth('2')

            plt.subplots_adjust(top=4.8, right = 4.7,wspace = 0.03,hspace = 0.2)


        # If >6 libraries are chosen and is not a multiple of 3, delete empty plots
        if len(np.unique(rows))*len(np.unique(cols)) != len(all_libraries):
            diff = (len(np.unique(rows))*len(np.unique(cols))) - len(all_libraries)
            for i in range (1,int(diff+1)):
                fig.delaxes(axes[rows[-i]][cols[-i]])

    # Save results 
    if plot_name != '':
        plot_name = str(plot_name + '_')
    plt.savefig(str(plot_name.replace(' ','_')+'enrichment_results_' + date.today().strftime("%Y%m%d") + '.'+fig_format.lower() ),bbox_inches = 'tight')
    
    # Show plot 
    plt.show()
    
    
               

In [ ]:
enrichr_figure(results[0],results[1],results[2],output_file_name, enrichr_libraries,figure_file_format)